## Estimate the multinomial logit (mnl) model for dc metro hts 2017/2018 survey using biogeme libraries

In [11]:
import pandas as pd
from matplotlib import pyplot as plt
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.version as ver
from biogeme.expressions import Beta

Check the version of Biogeme

In [12]:
ver.getVersion()

'3.2.6'

# Step 2: prepare the data

In [13]:
df = pd.read_csv('hts_data/trip_processed.csv')


C:\Users\makhijaa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (24,25,27,29,52,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Remove unwanted columns and filter hbw trips excluding outliers

In [14]:
df.drop(columns=['DEPARTURE_TIME_HHMM','ARRIVAL_TIME_HHMM','MPO_TRAVEL_MODE','MPO_TRAVEL_MODE_DETAIL','MPO_TRANSIT_ACCESS_MODE','MPO_TRANSIT_EGRESS_MODE','MPO_SUBWAY_ACCESS_MODE','MPO_SUBWAY_EGRESS_MODE'], axis=1, inplace=True) 
df_hbw = df[(df["PURP"]==1) & (df["INC"]==1) & (df["CHOICE"]>0) & (df["SPEED"]>3) & (df["SPEED"]<100) & (df["DISTANCE"]<=120)]

In [15]:
df_hbw.head()


,HOUSEHOLD_ID,PERSON_ID,PERSNO,TRIPID,TRIPNO,O_PURPOSE,O_ACTIVITY,MPO_O_ACTIVITY,O_STATE_FIPS,O_STATE_COUNTY_FIPS,...,ExBusCost,PURP,SovAV,HovAV,MetroAV,CrAV,BusAV,ExBusAV,SPEED,INC
12814,170001127,17000112701,1,1.700000e+12,1,1,1,1,24,24021,...,425,1,1,1,1,1,1,1,12.106953,1
12815,170001127,17000112701,1,1.700000e+12,2,2,2,3,24,24021,...,425,1,1,1,1,1,1,1,12.106953,1
12889,170001414,17000141401,1,1.700000e+12,1,1,1,1,51,51059,...,625,1,1,1,1,1,1,1,26.090401,1
12890,170001414,17000141401,1,1.700000e+12,2,3,2,4,51,51510,...,625,1,1,1,1,1,1,1,43.484002,1
12891,170001414,17000141401,1,1.700000e+12,3,1,1,2,51,51059,...,625,1,1,1,1,1,1,1,19.834972,1


In [16]:
database = db.Database("hbw",df_hbw)

## Define the name of the variables as Python variables

In [17]:
globals().update(database.variables)

## Remove some observations

In [18]:
database.getSampleSize()

1562

# Model specification - simple mnl 6 modes with tt and cost parameters


Create parameters to be estimated

Beta

    1.name of parameter
    2.default value for the parameter
    3.lower bound
    4.upper bound
    5.flag indicating if parameter is to be estimated


In [19]:
B_TIME    = Beta('B_TIME',0,None, None ,0)
B_COST    = Beta('B_COST',0,None ,None ,0)

## Definition of new variables

## Specification of the utility functions

In [20]:
V1 = B_TIME * SovTT + \
     B_COST * SovCost
V2 = B_TIME * HovTT + \
     B_COST * HovCost
V3 = B_TIME * MetroTT + \
     B_COST * MetroCost
V4 = B_TIME * CrTT + \
     B_COST * CrCost
V5 = B_TIME * BusTT + \
     B_COST * BusCost
V6 = B_TIME * ExBusTT + \
     B_COST * ExBusCost


## Associate the utility functions with the numbering of the alternatives

In [21]:
V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6}

## Associate the availability conditions with the alternatives

In [22]:
av = {1: 1,
      2: 1,
      3: 1,
      4: 1,
      5: 1,
      6: 1}

## The contribution to the log likelihood function is the logarithm of a logit model

In [23]:
logprob = models.loglogit(V, av, CHOICE)

# Biogeme

In [24]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'hts_2018_hbwmnl'

## Running the estimation

In [25]:
results = biogeme.estimate()

## Read the results

In [26]:
pandasResults = results.getEstimatedParameters()
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
B_COST,-0.004589,0.000195,-23.515149,0.0,0.000367,-12.487596,0.0
B_TIME,-0.040523,0.002201,-18.410120,0.0,0.003639,-11.137155,0.0


In [27]:
gs = results.getGeneralStatistics()

for k,v in gs.items():
    print("{}= {}".format(k.ljust(45),v[0]))

Number of estimated parameters               = 2
Sample size                                  = 1562
Excluded observations                        = 0
Init log likelihood                          = -2798.7282909342216
Final log likelihood                         = -2158.891995301038
Likelihood ratio test for the init. model    = 1279.672591266367
Rho-square for the init. model               = 0.22861679631630294
Rho-square-bar for the init. model           = 0.22790218603902856
Akaike Information Criterion                 = 4321.783990602076
Bayesian Information Criterion               = 4332.491435262876
Final gradient norm                          = 0.0007768460029123466
Nbr of threads                               = 12
